# Model: Decision Tree

## Date: Nov 23, 2023

---

## Introduction

For the next model, a decision tree will be used. This gives a good variety as log reg is probability based, svm distance based. It also handles non linearity well compared to linear SVM, log reg, and is a much quicker algorithm for large datasets compared to svm's. Since the last few model accuracies stagnated, there might be some non linearity in the data they are not capturing well. Therefore a decision tree is a good next model, however they can be prone to overfitting.  

Unlike with the previous models, the data does not need to be scaled. As it is nonlinear, the linear assumptions dont need to be satisfied. This results in not having to check and remove features with high colinearity and multicollinearity, and therefore more features will be included in this model compared to the previous models. However, this is an advantage of decision trees, so as long this caveat is kept in mind, the comparison can still be made. 

The hyperparameters that can be used:  
1. max_depth: The maximum depth of the tree. This controls overfitting as higher depth allows the model to discern more and more
2. min_samples_split: The minimum number of samples required to split an internal node. Higher values prevent the model from learning too much detail.
3. 
min_samples_leaf: The minimum number of samples required to be at a leaf node. Similar to min_samples_split, it controls overfittin 
4. 
max_features: The number of features to consider when looking for the best sp- 
5. .
max_leaf_nodes: Limits the number of leaf nodes in the tree, helping to control overfiig.  
6. g.
criterion: The function used to measure the quality of a split. Common criteria include 'gini' for Gini Impurity and 'entropy' for Information G
7. min_impurity_decrease: The minimum decrease in inpurity for the next split to be performed.  
  
 Note: There was a visualization error with the confusion matrices. The matrix is also printedain.

--------

### Table of Contents

1. [Introduction](#Introduction)
   - [Table of Contents](#Table-of-contents)
   - [Import Librarys](#Import-Librarys)
   - [Data Dictionary](#Data-Dictionary)
   - [Define Functions](#Define-Functions)
   - [Load the data](#Load-the-data)
3. [Decision Tree](#Decision-Tree)
   - [1st Iteration](#1st-Iteration)
   - [2nd Iteration](#2nd-Iteration)
   - [3rd (Final) Iteration](#3rd-(Final)-Iteration)
4. [Random Forest](#Random-Forest)
5. [Conclusion](#Conclusion)


### Import Librarys

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, ConfusionMatrixDisplay
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils import resample
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestClassifier

import shap

from pathlib import Path
from joblib import dump
from helpers import display_corr_heatmap, data_dict

### Data Dictionary

In [ ]:
data_dict()

### Load the Data

In [ ]:
# Define the relative path to the file
parquet_file_path = Path('../Data/Lending_club/model_cleaned')

try:
    # Read the parquet file
    loans_df = pd.read_parquet(parquet_file_path)
except FileNotFoundError as e:
    print(e.args[1])
    print('Check file location')

In [ ]:
loans_df.head()

### Decision Tree model

### Preprocessing

***Train test split***

In [ ]:
# Split the data
X = loans_df.drop(columns=['loan_status'], inplace=False)
y = loans_df['loan_status']

# Split into train and test sets. Stratify to ensure any inbalance is preserved as in the original data. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=11, stratify=y)

***Data Inbalance***

Decision trees are still sensitive to data inbalance, so the data will be balanced again.

In [ ]:
print('Number of class 1 examples before:', X_train[y_train == 1].shape[0])

# Downsample majority class
X_downsampled, y_downsampled  = resample(X_train[y_train == 1],
                                   y_train[y_train == 1],
                                   replace=False,
                                   n_samples=X_train[y_train == 0].shape[0],
                                   random_state=1)

print('\nNumber of class 1 examples after:', X_downsampled.shape[0])

# Combine the downsampled successful loans with the failed loans. Will keep as a df since changing to 
X_train_bal = pd.concat([X_train[y_train == 0], X_downsampled])
y_train_bal = np.hstack((y_train[y_train == 0], y_downsampled))

print("New X_train shape: ", X_train_bal.shape)
print("New y_train shape: ", y_train_bal.shape)
print("X_test shape: ", X_test.shape)
print("y_test shape: ", y_test.shape)

No features need to be dropped so there are now 25 extra features that were not available in log reg and svm

***Inspect Categorical Features***

Categorical features have to be numerically encoded. The encodings will be the same as the other notebooks.

In [ ]:
categorical_columns = X_train_bal.select_dtypes('object').columns.tolist()
display(categorical_columns)
categorical_columns.remove('verification_status')

In [ ]:
# Instantiate
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
ordinal_transformer = OrdinalEncoder(categories=[['Not Verified', 'Source Verified', 'Verified']])

# Combine into a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, ['home_ownership', 'verification_status', 'purpose', 'application_type']),
        ('ord', ordinal_transformer, ['verification_status'])],
    remainder='passthrough',
    n_jobs=2
)

# Fit to the train set
preprocessor.fit(X_train_bal)

# Transform the train and test sets
X_train_transformed = preprocessor.transform(X_train_bal)
X_test_transformed = preprocessor.transform(X_test)

print("Shape of train transformed: ", X_train_transformed.shape)
print("Shape of test transformed: ", X_test_transformed.shape)

## Decision Tree

### 1st Iteration

Since there are so many hyperparameters for decision trees, it does not make sense to manually iterate one by one to find acceptable hyperparameter ranges for the final iteration, like what was done for log reg and svm. Instead, a randomized gridsearch, which simply computes random combinations of hyperparameters, is more efficent. This works like the GridsearchCV function seen in the other notebooks, however it simply takes a random sample of hyperparamter combinations, as opposed to an exhaustive search. This saves on computations and time.

***Run the first iteration***

In [ ]:
%%time
# Instantiate decision tree
dt_classifier = DecisionTreeClassifier(random_state=1)

# Hyperparameters grid to search
param_dist = {
    "criterion": ["gini", "entropy"],
    "max_depth": [None, 7, 8, 9],
    "min_samples_split": [4,5,6],
    "min_samples_leaf": [9, 10, 11],
    "max_features": [None, "sqrt", "log2"]
}

# Define scoring metrics
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='weighted'),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1': make_scorer(f1_score, average='binary', pos_label=1) #f1 score for class 1
}

# Randomized Grid Search with cross-validation
random_search = RandomizedSearchCV(
    dt_classifier, 
    param_distributions=param_dist, 
    n_iter=100, #pull out 100 combinations
    scoring=scoring, 
    refit='f1', #refit on f1 for class 1
    cv=5, #5 fold cross validation
    random_state=1, 
    verbose=10,
    n_jobs=3 #adjust based on your cpu cores
)

# Fit
random_search.fit(X_train_transformed, y_train_bal)

# Best model
best_dt_model = random_search.best_estimator_

# Best hyperparameters
best_params = random_search.best_params_

In [ ]:
# Best hyperparameters
best_params = random_search.best_params_
print(f"Best parameters: {best_params}")

***Score the first iteration***

In [ ]:
# Make predictions on both training and test data
y_pred_train = best_dt_model.predict(X_train_transformed)
y_pred_test = best_dt_model.predict(X_test_transformed)

# Calculate accuracies for train and test sets
train_accuracy = accuracy_score(y_train_bal, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)

# Print the train and test accuracies
print(f"Train Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")

# Check for any overfitting
if train_accuracy > test_accuracy:
    print("The model might be overfitting.")
else:
    print("The model seems to generalize well.")

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred_test)

# Classification Report
print("Classification Report:")
print(classification_report(y_test, y_pred_test))
print("Confusion maxtrix")
print(conf_matrix)

***1st model Interpretation***

For the first iteration, there is noticeable overfitting. The model underperformed relative to the baseline log reg model, even though it was observed that there were many non linear relationships. The weighted average F1 score, a good overall metric, is lower than log reg's 0.69.
Overall, the model overfit, and underperformed. A tree depth of 8 is quite high and will be adjusted next to control the overfitting. Further interpretation will be done once the model generalizes well. 

***1st model Feature importance***

For log reg, the coefficients gave information on how much predictability a feature had. With a decision tree, feature importance is calculated based on the features ability to reduced impurity, ie segment the data into "pure" buckets. Impurity is a measure of the homogeneity of the labels at a node. Features that are able to greatly reduce impurity, ie are great discerners for the different classes, are given a high importance. 

In [ ]:
# Get feature names
feature_names_transformed = preprocessor.get_feature_names_out()
print(type(feature_names_transformed))

# Get importances 
importances = best_dt_model.feature_importances_

# Sort and get indices
indices = np.argsort(importances)[::-1]

# PLot
plt.figure(figsize=(10, 12))
plt.title("Feature importances")
plt.barh(range(X_train_transformed.shape[1]), importances[indices], color="r")
plt.yticks(range(X_train_transformed.shape[1]), feature_names_transformed[indices])
plt.xlabel('Importance')
plt.ylabel('Features')
plt.tight_layout()
plt.show()

Interest rate has the highest importance by far. This was initially worrisome, however, interest rate is not a leaky feature. Instead, it is possible that the larger financial burden brought on by a larger interest rate is enough to force a loan default. However, this is purely speculation and investigated below.

***Tree Diagram***

Due to the depth, the tree will be truncated at a layer of 4 for visualization.  
https://mljar.com/blog/visualize-decision-tree/

In [ ]:
# Convert to a list
feature_names_list = feature_names_transformed.tolist()

plt.figure(figsize=(20,10))
plot_tree(best_dt_model, 
          feature_names=feature_names_list, 
          class_names=['Class 0', 'Class 1'], 
          filled=True, 
          rounded=True, 
          fontsize=12,
          max_depth=4)
plt.show()

The root is split based off of interest rate, the highest importance feature. This split makes sense, as seen in EDA, the median interest rate for failed loans was 15% whereas for successful loans it as 12%, making it a good discriminator. For failed loans, the next features were the loan term and annual income. 

### 2nd Iteration

Now that there is a baseline decision tree model, we can now vary max depth manually to address the overfitting. Max depth is the most important parameter for controlling overfitting, as it places a limit on how many times the tree can split, thus increasing its purity. Limiting max depth stops the tree from over capturing the noise on the train set, allowing it to better generalize to other data. The previous best model will be used, and the C value iterated. 

***Run the model 2nd Iteration***

In [ ]:
%%time
# Define lists for accuracies to be plotted
train_accuracies = []
test_accuracies = []
f1_scores_train = []
f1_scores_test = []

# Depth of 10 to 2 backwards 
for depth in range(10, 1, -1):
    
    # Set max_depth
    best_dt_model.set_params(max_depth=depth)
    
    # Fit 
    best_dt_model.fit(X_train_transformed, y_train_bal)
    
    # Predictions
    y_pred_train = best_dt_model.predict(X_train_transformed)
    y_pred_test = best_dt_model.predict(X_test_transformed)
    
    # Calculate accuracies
    train_accuracy = accuracy_score(y_train_bal, y_pred_train)
    test_accuracy = accuracy_score(y_test, y_pred_test)
    
    # Calculate F1 scores
    f1_train = f1_score(y_train_bal, y_pred_train, average='binary', pos_label=1)
    f1_test = f1_score(y_test, y_pred_test, average='binary', pos_label=1)
    
    # Append to lists
    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)
    f1_scores_train.append(f1_train)
    f1_scores_test.append(f1_test)
    
    # Print the results for this iteration
    print(f"Max depth: {depth}")
    print(f"Train Accuracy: {train_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Train F1 Score: {f1_train:.4f}")
    print(f"Test F1 Score: {f1_test:.4f}")
    print("-" * 30)

***2nd model interpretation***

Since the 2nd iteration only focus's on addressing the overfitting, only the train and test scores really need to be evaluated. 

In [ ]:
depths = list(range(10, 1, -1))

plt.figure(figsize=(14, 7))

# Plot train and test accuracies
plt.subplot(1, 2, 1)
plt.plot(depths, train_accuracies, marker='o', label='Train Accuracy')
plt.plot(depths, test_accuracies, marker='*', label='Test Accuracy')
plt.xlabel('Max Depth')
plt.ylabel('Accuracy')
plt.title('Accuracies by Max Depth')
plt.legend()
plt.gca().invert_xaxis()  # Invert x-axis

# Plot F1 scores
plt.subplot(1, 2, 2)
plt.plot(depths, f1_scores_train, marker='o', label='Train F1 Score for class 1')
plt.plot(depths, f1_scores_test, marker='*', label='Test F1 Score for class 1')
plt.xlabel('Max Depth')
plt.ylabel('F1 Score')
plt.title('Train/Test F1 Scores by Max Depth for class 1')
plt.legend()
plt.gca().invert_xaxis()

plt.show()

A depth of 4 seems optimal. The train and test accuracies converge, and the f1 score was the highest for the test set. Below a depth of 4 the tree is too shallow to perform well, and above 4 the model overfits and fails to generalize well to the test set. The model also achieves the highest f1 score as well at a depth of 4. 

### 3rd (Final) Iteration

Having established a good range for a few hyperparameters, we can now run an exhaustive search for the best combination. Recall the best parameters found were:  


- 'criterion':entropy    
- 'min_samples_split': 65
- 'min_samples_leaf': 10
- 'max_features': None
- 'max_depth':84  
Max depth will be fixed at 4, as this was found to be optimal, however it is not the only hyperparameter that has a large impact on the models fit. Other hyperparameters also control the models fit, so fixing max depth to 4 does not automatically equate to a possible poor fitting model or guarantee no model overfitting. 

The previous models were fit on class 1 f1 scores, in order to give a range that would lead to a better fitting model overall, and not too narrow if only optimized for precision. However, unlike log reg, there is no threshold that can be modified after the fact to favor precision or recall, so the final model will now be optimized for class 1 precision.


More information on features to control overfitting and important hyperparameters:  
https://blog.dataiku.com/narrowing-the-search-which-hyperparameters-really-matter  
https://medium.com/@ompramod9921/decision-trees-8e2391f93fa7

***Run the model 3rd (Final) Iteration***

In [ ]:
%%time
# Instantiate
dt_classifier = DecisionTreeClassifier(random_state=1)

# Hyperparameters
param_grid = {
    
    # Keep fixed
    "criterion": ["entropy"],
    "max_depth": [4],

    # Increase to reduce overfitting
    "min_samples_split": [7,8],
    "min_samples_leaf": [11,12],
}
#Best parameters from last iteration: {'min_samples_split': 6, 'min_samples_leaf': 10, 'max_features': None, 'max_depth': 8, 'criterion': 'entropy'}

# Define scoring metrics
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, pos_label=1, average='weighted'),
    'recall': make_scorer(recall_score, pos_label=1, average='weighted'),
    'f1': make_scorer(f1_score, pos_label=1, average='binary')
}

# Grid Search with cross-validation
grid_search = GridSearchCV(
    dt_classifier, 
    param_grid=param_grid, 
    scoring=scoring, 
    refit='precision', # Refit on precision score for class 1 now
    cv=5, 
    verbose=10,
    n_jobs=3)

# Perform the grid search
grid_search.fit(X_train_transformed, y_train_bal)

# Best model
best_dt_model = grid_search.best_estimator_

# Best hyperparameters
best_params = grid_search.best_params_

# Print the best hyperparameters
print("Best hyperparameters:")
print(best_params)

***Score the model 3rd (Final) Iteration***

In [ ]:
# Make predictions using the training set and the test set
y_train_pred = best_dt_model.predict(X_train_transformed)
y_test_pred = best_dt_model.predict(X_test_transformed)

# Calculate accuracies for train and test sets
train_accuracy = accuracy_score(y_train_bal, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

# Print the train and test accuracies
print(f"Train Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")

# Classification report
print("Classification Report:")
print(classification_report(y_test, y_test_pred))

# Confusion matrix
conf_matrix_test = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_test, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix for the Test Set')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()


# AUC score and ROC curve for the test set
y_test_proba = best_dt_model.predict_proba(X_test_transformed)[:, 1]
auc_score = roc_auc_score(y_test, y_test_proba)
fpr, tpr, _ = roc_curve(y_test, y_test_proba)
print(conf_matrix_test)
print('AUC score: ', auc_score)

For the final iteration, we saw a large improvement. The train and test scores are very similar, showing there is no longer any overfitting. Both min_samples_split and min_samples_leaf were increased to help with the issue. However, no improvement was made over the baseline log reg model. For class 1, the same precision of 0.87 was achieved, with a 0.02 decrease in recall. Similarly, for class 0, the precision and recall were slightly worse compared to the baseline. Overall, the weighted average f1 score and AUC dropped by 0.02.    

***3rd model Feature Importance***

In [ ]:
# Get feature names
feature_names_transformed = preprocessor.get_feature_names_out()
print(type(feature_names_transformed))

# Get importances 
importances = best_dt_model.feature_importances_

# Sort and get indices
indices = np.argsort(importances)[::-1]

# PLot
plt.figure(figsize=(10, 12))
plt.title("Feature importances")
plt.barh(range(X_train_transformed.shape[1]), importances[indices], color="r")
plt.yticks(range(X_train_transformed.shape[1]), feature_names_transformed[indices])
plt.xlabel('Importance')
plt.ylabel('Features')
plt.tight_layout()
plt.show()

Once again, the model is very concentrated on what features is relies on. Interest rate is of the highest. 

***3rd model Tree Diagram***

In [ ]:
# Convert to a list
feature_names_list = feature_names_transformed.tolist()

plt.figure(figsize=(28,14))
plot_tree(best_dt_model, 
          feature_names=feature_names_list, 
          class_names=['Class 0', 'Class 1'], 
          filled=True, 
          rounded=True, 
          fontsize=12,
          max_depth=4)
plt.show()

***3rd model SHAP values***

SHAP values are based on game theory and assign an importance value to each feature in a model. Features with positive SHAP values positively impact the prediction, while those with negative values have a negative impact. The magnitude is a measure of how strong the effect is.  

It has a similar function to the feature importance plot above, however, it gives more infomation as well. 
  
https://www.datacamp.com/tutorial/introduction-to-shap-values-machine-learning-interpretability

In [ ]:
%%time
# https://www.kaggle.com/code/dansbecker/shap-values
# Instantiate
explainer = shap.TreeExplainer(best_dt_model)

# Calculate SHAP values
shap_values = explainer(X_test_transformed)

# Selects the SHAP values for the positive class
shap_values_positive_class = shap_values[..., 1] 

# Plot
shap.summary_plot(shap_values_positive_class, X_test_transformed, feature_names=feature_names_transformed)

The summary plot shows the contribution of each feature to the model, for each sample. Each dot refers to a specific loan in the dataset. The color represents the value of the feature, ie, blue indicating lower feature values, red higher. Where on the x axis the dot lies indicates if it was a positive or negative impact, and the magnitude. SHAP values range from -1 to +1.  If there are many dots overlapping, they are seperated sligtly vertically. These plots give far more information compared to the feature importances calculated a few cells above.

For the `int_rate` feature, higher interest rates (in red) tend to have more negative shap values, indicating it pushes the prediction towards the negative class (failed loans). The vice versa is also true; a lower interest rate pushes prediction towards the positive class. Although the color gradient left to right is consistent, there are groupings of interest rates at different shap values.

***Save the 3rd (Final) Iteration***

In [ ]:
#relative path to models folder
model_file_path = Path('../Models/best_decision_tree_model.joblib')
dump(best_dt_model, model_file_path)

## Random Forest

We can now use a ensemble of decision trees called random forest. It uses the bootstrapping technique. A random sample of the dataset is taken, with replacement, and then a tree is trained on each sample. The theory is that, since the trees are trained on different datasets and that decision trees in general are quite sensitive to small changes in the data, diversity is introduced into the model which helps with the overfitting we experienced above. Max features will also be introduced 

This forest of trees are then aggregated back together for predictions. The consensus protocol is usually a simple majority vote among the trees for classification problems. 

In addition to the hyperparameters for a single decision tree, random forest has:
- n_estimators: The number of trees in the forest. Generally, a larger number of trees increases performance but at a computational cost.

***Fit the model***

In [ ]:
%%time
# Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=1)

# Hyperparameters grid to search
forrest_param_grid = {
    # Keep fixed
    "criterion": ["entropy"],
    "max_depth": [4,5],

    # Increase to reduce overfitting
    "min_samples_split": [7, 8],
    "min_samples_leaf": [12, 13],

    # Parameters for Random Forest
    "n_estimators": [150],  # Number of trees in the forest. 100 is default
    "max_features": ['sqrt', 'log2'],  # Number of features to consider at every split
    #max_features{“sqrt”, “log2”, None}, int or float, default=”sqrt”

}

# Define scoring metrics
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, pos_label=1, average='binary'),
    'recall': make_scorer(recall_score, pos_label=1, average='binary'),
    'f1': make_scorer(f1_score, pos_label=1, average='binary')
}

# Grid Search with cross-validation
grid_search = GridSearchCV(
    rf_classifier, 
    param_grid=forrest_param_grid, 
    scoring=scoring, 
    refit='precision',  # Refit on precision score for class 1
    cv=5, 
    verbose=10,
    n_jobs=3  # Using -1 to use all processors
)

# Perform the grid search
grid_search.fit(X_train_transformed, y_train_bal)

# Best model
best_rf_model = grid_search.best_estimator_

# Best hyperparameters
best_params = grid_search.best_params_

# Print the best hyperparameters
print("Best hyperparameters:")
print(best_params)

***Score the model***

In [ ]:
y_train_pred = best_rf_model.predict(X_train_transformed)
y_test_pred = best_rf_model.predict(X_test_transformed)

# Calculate accuracies for train and test sets
train_accuracy = accuracy_score(y_train_bal, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Train Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")

if train_accuracy > test_accuracy:
    print("The model might be overfitting.")
else:
    print("The model seems to generalize well.")

# Classification Report
print("Classification Report:")
print(classification_report(y_test, y_test_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()
print(conf_matrix)

# Calculate scores
y_test_proba = best_rf_model.predict_proba(X_test_transformed)[:, 1]
auc_score = roc_auc_score(y_test, y_test_proba)
fpr, tpr, _ = roc_curve(y_test, y_test_proba)

# Plot confusion matrix
plt.figure()
plt.plot(fpr, tpr, color='darkorange', label=f'ROC curve (area = {auc_score:.2f})')
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

The model's class 1 precision increased from 0.87 to 0.88, however, there is some slight overfitting. On the otherhand, recall dropped by 0.02, with the AUC staying constant. This is the best fitting model, in terms of precision, so far. 

***Save the Random Forest model***

In [ ]:
#relative path to models folder
model_file_path = Path('../Models/best_random_forest_model.joblib')
dump(best_rf_model, model_file_path)

***RF model SHAP values***

In [ ]:
%%time
feature_names_transformed = preprocessor.get_feature_names_out()

# Instantiate
explainer = shap.TreeExplainer(best_rf_model)

# Calculate SHAP values
shap_values = explainer(X_test_transformed)

shap_values_positive_class = shap_values[..., 1]

# For a global understanding, use the summary plot for the positive class
shap.summary_plot(shap_values_positive_class, X_test_transformed, feature_names=feature_names_transformed)

Interest rate is once again a good predictor. Shorter loan terms have a positive impact on the prediction, similar to the debt to income ratio. A low value in the label-encoded `cat__verification_status_not verified` means the loan has some verification and the feature has a positive impact on the prediction. These examples show how powerful the shap plots are, as they show different points in the features distribution effects the model prediction This is incredibly important to both the investor and borrower as the investor can better justify the decision for extending or withholding credit, increasing both accountability and transparancy. Futhermore, it allows the investor to see how exactly the features are influencing the model, and therefore is make the decision on keeping, removing, or engineering the feature to better.  

The shap plot for the random forest shows a wider distribution of important features compared to that of a single tree. For the single best optimized tree, interest rate was the only really important feature. However, for random forest there are many. This variety results in a model that is less dependent on a single features value, resulting in a more stable model. 

### Conclusion

In this notebook, we explored fitting decision trees and a random forest to our data. For decision trees, a random gridsearch was conducted to get ballpark ranges for hyperparameter values. However, since the model was overfitting, the max depth was manually varied and explored. A depth of 4 was shown to be optimal, achieving high metrics without excessive overfitting. An exhaustive gridsearch was then done based on the previous 2 iterations. This final model showed a large improvement over the previous iteration, however, no improvement over the baseline log reg was made. Finally, a random forest model was fit to the data. This model showed almost exactly the same metrics, however the model is far more stable and less dependent on only a few features. 